# Challenge 4 study - Quantized Model with First Layer on pre-processor (SW)

## Import libraries and data, init virtual device

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, ReLU

from akida import Device, AKD1000
import cnn2snn

device = AKD1000()

2022-11-10 22:54:48.324360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 22:54:48.804944: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 22:54:48.804984: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 22:54:48.874556: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 22:54:52.519459: W tensorflow/stream_executor/platform/de

## Data Preparation

In [2]:
coords_ch4 = np.load('coords_ch4.npy')
color = np.load('result_ch4.npy')

# Pack in a dataframe and scale for 8-bits quantization (first layer will be quantized on 8-bits)
data = pd.DataFrame(coords_ch4, columns = ['x', 'y'])
data = round(255*data).astype(np.uint8)

# Shuffle and split train/test
data['color'] = color
data_train = data.sample(frac=0.7, axis=0)
data_test = data.drop(data_train.index)

# Split x y
x_train = data_train.drop('color', axis=1)
y_train = data_train['color']
x_test = data_test.drop('color', axis=1)
y_test = data_test['color']
data

,x,y,color
0,219,139,1
1,170,68,0
2,113,147,0
3,202,249,0
4,224,54,1
...,...,...,...
9995,242,192,0
9996,138,16,1
9997,81,114,1
9998,95,238,1


## Model creation with Keras

In [3]:
base = Sequential(name="Base_ch4")
base.add(Dense(1000, input_shape=(2,), name='FC1'))
base.add(ReLU(name='relu1'))
base.add(Dense(500, input_shape=(2,), name='FC2'))
base.add(ReLU(name='relu2'))
base.add(Dense(250, input_shape=(2,), name='FC3'))
base.add(ReLU(name='relu3'))
base.add(Dense(100, input_shape=(2,), name='FC4'))
base.add(ReLU(name='relu4'))
base.add(Dense(50, input_shape=(2,), name='FC5'))
base.add(ReLU(name='relu5'))
base.add(Dense(1, input_shape=(2,), name='FC6'))
base.add(Activation('sigmoid', name='sigmoid'))

#base.summary()

## Model quantization (4-bits, input: 8-bit)

In [4]:
qmodel = cnn2snn.quantize(base, weight_quantization=4, activ_quantization=4, input_weight_quantization=8)
qmodel.summary()

print('\nAkida Compatibility:', cnn2snn.check_model_compatibility(qmodel, input_is_image=False))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 FC1 (QuantizedDense)        (None, 1000)              3000      
                                                                 
 relu1 (QuantizedReLU)       (None, 1000)              0         
                                                                 
 FC2 (QuantizedDense)        (None, 500)               500500    
                                                                 
 relu2 (QuantizedReLU)       (None, 500)               0         
                                                                 
 FC3 (QuantizedDense)        (None, 250)               125250    
                                                                 
 relu3 (QuantizedReLU)       (None, 250)               0         
                                                                 
 FC4 (QuantizedDense)        (None, 100)              

## Preview akida model

In [5]:
akmodel = cnn2snn.convert(qmodel, input_is_image=False)
akmodel.map(device)
akmodel.summary()

                   Model Summary                   
___________________________________________________
Input shape  Output shape  Sequences  Layers  NPs
[1, 1, 2]    [1, 1, 1]     2          7       5  
___________________________________________________

____________________________________________________
Layer (type)  Output shape  Kernel shape       NPs

================= SW/FC1 (Software) ================

FC1 (Fully.)  [1, 1, 1000]  (1, 1, 2, 1000)    N/A

==== HW/FC2-FC6 (Hardware) - size: 520116 bytes ====

FC2 (Fully.)  [1, 1, 500]   (1, 1, 1000, 500)  1  
____________________________________________________
FC3 (Fully.)  [1, 1, 250]   (1, 1, 500, 250)   1  
____________________________________________________
FC4 (Fully.)  [1, 1, 100]   (1, 1, 250, 100)   1  
____________________________________________________
FC5 (Fully.)  [1, 1, 50]    (1, 1, 100, 50)    1  
____________________________________________________
FC6 (Fully.)  [1, 1, 1]     (1, 1, 50, 1)      1  
___________

## Train quantized model

In [6]:
qmodel.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
            )
qmodel.fit(x_train, y_train, batch_size=64, epochs=600)
qmodel.evaluate(x_test, y_test)

Epoch 1/600
110/110 [==============================] - 23s 106ms/step - loss: 0.7752 - accuracy: 0.5214
Epoch 2/600
110/110 [==============================] - 9s 82ms/step - loss: 0.6735 - accuracy: 0.5631
Epoch 3/600
110/110 [==============================] - 9s 86ms/step - loss: 0.6856 - accuracy: 0.5556
Epoch 4/600
110/110 [==============================] - 12s 109ms/step - loss: 0.6625 - accuracy: 0.5706
Epoch 5/600
110/110 [==============================] - 11s 102ms/step - loss: 0.6258 - accuracy: 0.5890
Epoch 6/600
110/110 [==============================] - 11s 103ms/step - loss: 0.6178 - accuracy: 0.6036
Epoch 7/600
110/110 [==============================] - 11s 104ms/step - loss: 0.5966 - accuracy: 0.6283
Epoch 8/600
110/110 [==============================] - 12s 106ms/step - loss: 0.6010 - accuracy: 0.6176
Epoch 9/600
110/110 [==============================] - 11s 95ms/step - loss: 0.5956 - accuracy: 0.6293
Epoch 10/600
110/110 [==============================] - 10s 87ms/step

110/110 [==============================] - 12s 105ms/step - loss: 0.2400 - accuracy: 0.8910
Epoch 80/600
110/110 [==============================] - 13s 117ms/step - loss: 0.2278 - accuracy: 0.8974
Epoch 81/600
110/110 [==============================] - 11s 100ms/step - loss: 0.2312 - accuracy: 0.9027
Epoch 82/600
110/110 [==============================] - 11s 96ms/step - loss: 0.2420 - accuracy: 0.8937
Epoch 83/600
110/110 [==============================] - 11s 103ms/step - loss: 0.2345 - accuracy: 0.8959
Epoch 84/600
110/110 [==============================] - 11s 98ms/step - loss: 0.2248 - accuracy: 0.9070
Epoch 85/600
110/110 [==============================] - 11s 99ms/step - loss: 0.2269 - accuracy: 0.9031
Epoch 86/600
110/110 [==============================] - 11s 99ms/step - loss: 0.2254 - accuracy: 0.9000
Epoch 87/600
110/110 [==============================] - 11s 100ms/step - loss: 0.2369 - accuracy: 0.9006
Epoch 88/600
110/110 [==============================] - 11s 98ms/step - 

110/110 [==============================] - 12s 105ms/step - loss: 0.2036 - accuracy: 0.9129
Epoch 158/600
110/110 [==============================] - 11s 103ms/step - loss: 0.2188 - accuracy: 0.9071
Epoch 159/600
110/110 [==============================] - 11s 104ms/step - loss: 0.1964 - accuracy: 0.9164
Epoch 160/600
110/110 [==============================] - 11s 103ms/step - loss: 0.2303 - accuracy: 0.9016
Epoch 161/600
110/110 [==============================] - 11s 97ms/step - loss: 0.2093 - accuracy: 0.9149
Epoch 162/600
110/110 [==============================] - 11s 104ms/step - loss: 0.2170 - accuracy: 0.9116
Epoch 163/600
110/110 [==============================] - 11s 100ms/step - loss: 0.1929 - accuracy: 0.9219
Epoch 164/600
110/110 [==============================] - 11s 101ms/step - loss: 0.2113 - accuracy: 0.9087
Epoch 165/600
110/110 [==============================] - 11s 104ms/step - loss: 0.1929 - accuracy: 0.9186
Epoch 166/600
110/110 [==============================] - 11s 

110/110 [==============================] - 11s 98ms/step - loss: 0.1950 - accuracy: 0.9189
Epoch 235/600
110/110 [==============================] - 11s 98ms/step - loss: 0.2142 - accuracy: 0.9087
Epoch 236/600
110/110 [==============================] - 11s 100ms/step - loss: 0.1939 - accuracy: 0.9217
Epoch 237/600
110/110 [==============================] - 11s 99ms/step - loss: 0.1916 - accuracy: 0.9161
Epoch 238/600
110/110 [==============================] - 11s 97ms/step - loss: 0.2091 - accuracy: 0.9097
Epoch 239/600
110/110 [==============================] - 11s 99ms/step - loss: 0.1801 - accuracy: 0.9240
Epoch 240/600
110/110 [==============================] - 11s 96ms/step - loss: 0.2051 - accuracy: 0.9119
Epoch 241/600
110/110 [==============================] - 11s 98ms/step - loss: 0.1825 - accuracy: 0.9227
Epoch 242/600
110/110 [==============================] - 10s 93ms/step - loss: 0.1942 - accuracy: 0.9181
Epoch 243/600
110/110 [==============================] - 11s 98ms/st

110/110 [==============================] - 10s 95ms/step - loss: 0.1870 - accuracy: 0.9184
Epoch 313/600
110/110 [==============================] - 11s 99ms/step - loss: 0.2024 - accuracy: 0.9130
Epoch 314/600
110/110 [==============================] - 11s 98ms/step - loss: 0.1858 - accuracy: 0.9186
Epoch 315/600
110/110 [==============================] - 11s 98ms/step - loss: 0.1879 - accuracy: 0.9200
Epoch 316/600
110/110 [==============================] - 11s 99ms/step - loss: 0.1910 - accuracy: 0.9186
Epoch 317/600
110/110 [==============================] - 11s 98ms/step - loss: 0.2113 - accuracy: 0.9143
Epoch 318/600
110/110 [==============================] - 10s 95ms/step - loss: 0.1866 - accuracy: 0.9199
Epoch 319/600
110/110 [==============================] - 11s 97ms/step - loss: 0.1992 - accuracy: 0.9133
Epoch 320/600
110/110 [==============================] - 11s 98ms/step - loss: 0.1831 - accuracy: 0.9210
Epoch 321/600
110/110 [==============================] - 10s 90ms/ste

110/110 [==============================] - 11s 96ms/step - loss: 0.2048 - accuracy: 0.9110
Epoch 391/600
110/110 [==============================] - 11s 102ms/step - loss: 0.1889 - accuracy: 0.9200
Epoch 392/600
110/110 [==============================] - 11s 98ms/step - loss: 0.2022 - accuracy: 0.9200
Epoch 393/600
110/110 [==============================] - 10s 95ms/step - loss: 0.1963 - accuracy: 0.9141
Epoch 394/600
110/110 [==============================] - 10s 95ms/step - loss: 0.1858 - accuracy: 0.9187
Epoch 395/600
110/110 [==============================] - 10s 95ms/step - loss: 0.1973 - accuracy: 0.9146
Epoch 396/600
110/110 [==============================] - 11s 99ms/step - loss: 0.1919 - accuracy: 0.9210
Epoch 397/600
110/110 [==============================] - 11s 100ms/step - loss: 0.2119 - accuracy: 0.9080
Epoch 398/600
110/110 [==============================] - 11s 97ms/step - loss: 0.1906 - accuracy: 0.9156
Epoch 399/600
110/110 [==============================] - 11s 99ms/s

Epoch 468/600
110/110 [==============================] - 10s 93ms/step - loss: 0.1847 - accuracy: 0.9227
Epoch 469/600
110/110 [==============================] - 10s 94ms/step - loss: 0.1880 - accuracy: 0.9166
Epoch 470/600
110/110 [==============================] - 10s 93ms/step - loss: 0.2079 - accuracy: 0.9091
Epoch 471/600
110/110 [==============================] - 10s 93ms/step - loss: 0.1897 - accuracy: 0.9179
Epoch 472/600
110/110 [==============================] - 10s 91ms/step - loss: 0.1933 - accuracy: 0.9187
Epoch 473/600
110/110 [==============================] - 10s 92ms/step - loss: 0.1874 - accuracy: 0.9193
Epoch 474/600
110/110 [==============================] - 10s 92ms/step - loss: 0.1914 - accuracy: 0.9159
Epoch 475/600
110/110 [==============================] - 11s 96ms/step - loss: 0.1928 - accuracy: 0.9136
Epoch 476/600
110/110 [==============================] - 10s 90ms/step - loss: 0.1941 - accuracy: 0.9150
Epoch 477/600
110/110 [==============================] 

110/110 [==============================] - 10s 90ms/step - loss: 0.1981 - accuracy: 0.9160
Epoch 547/600
110/110 [==============================] - 10s 95ms/step - loss: 0.1816 - accuracy: 0.9244
Epoch 548/600
110/110 [==============================] - 11s 98ms/step - loss: 0.1795 - accuracy: 0.9216
Epoch 549/600
110/110 [==============================] - 11s 98ms/step - loss: 0.1784 - accuracy: 0.9214
Epoch 550/600
110/110 [==============================] - 11s 97ms/step - loss: 0.2014 - accuracy: 0.9167
Epoch 551/600
110/110 [==============================] - 11s 96ms/step - loss: 0.1854 - accuracy: 0.9191
Epoch 552/600
110/110 [==============================] - 11s 99ms/step - loss: 0.1836 - accuracy: 0.9191
Epoch 553/600
110/110 [==============================] - 10s 95ms/step - loss: 0.1744 - accuracy: 0.9250
Epoch 554/600
110/110 [==============================] - 11s 99ms/step - loss: 0.1714 - accuracy: 0.9301
Epoch 555/600
110/110 [==============================] - 10s 95ms/ste

[0.24007077515125275, 0.9056666493415833]